In [1]:
import requests
from datetime import date
import ryan
import fire

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:
# Use a service account.
cred = credentials.Certificate('flightsAccount.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
#matching destinations
def matchDestinations(Origin, Arrival):
    matchingDestinations = []
    for destination in Origin:
        if destination in Arrival:
            matchingDestinations.append(d)
    return matchingDestinations

In [4]:
#imput:
trip = 'VNO-PSR'
Outbound = '2023-09-04'
Return = '2023-09-24'

In [6]:
#checking for destinations and airports
checkDate = int(str(date.today()).replace('-',''))
for origin in trip.split('-'):
    fireBatch = db.batch()
    #API call for destinations
    destinations = requests.get(ryan.getDestinations(origin,2)).json()
    #read response, set batch for Firestore and create array of destinations 
    rDestinations = ryan.readDestinations(destinations, fireBatch, db)
    #commiting batch
    fireBatch.commit()

    #create or update destinations from airport
    fire.fireDestinations(db, origin, rDestinations)

    #get array of matching destinations
    if origin == trip[:3]:
        rOriginDestinations = rDestinations
    else:
        rArrivalDestinations = rDestinations


#array for matched destinations
rConnections = []
if trip[:3] in rArrivalDestinations:
    isDirect = True
else:
    isDirect = False
for d in rOriginDestinations:
    if d in rArrivalDestinations:
        rConnections.append(d)
    
fire.fireConnections(db, trip, rConnections, isDirect)
fire.fireTrip(db, trip, int(str(date.today()).replace('-','')), True, rConnections, isDirect, Outbound.replace('-',''), Return.replace('-',''))
# fire.fireTrip(db, trip, int(str(date.today()).replace('-','')), True, rConnections, isDirect, Outbound.replace('-',''))

In [7]:
for i in rConnections:
    

BGY
HHN
MLA
STN
TRN
